In [1]:
import os 
os.chdir("../..")
import numpy
import numpy as np
import pandas as pd
import datetime
from scripts.PdfParse import *
from scripts.SupportFunc import *
from dtaidistance import dtw
from dtaidistance import dtw_visualisation as dtwvis
import random

In [2]:
fiji_folder = os.getcwd() + "/data/tourism/fiji/"
fiji_files = os.listdir(fiji_folder)
filepaths = [fiji_folder + file for file in fiji_files]

## Fiji Census

In [3]:
df = pd.read_csv(filepaths[-1]).drop("Unnamed: 0", axis=1)
df.head(5)

,year,dates,australia,newzealand,usa,canada,unitedkingdom,continentaleurope,japan,southkorea,china,india,hongkong,rest ofasia,pacificislands,others,total
0,2020,2020-01-01,29886,11316,6209,1075,1423,2802,726,316,4469,367,65,1238,5022,472,65386
1,2020,2020-02-01,16690,8242,7020,1167,1286,2524,1042,161,1661,469,35,1014,4685,347,46343
2,2020,2020-03-01,11260,5520,3835,623,723,1361,460,28,835,276,55,896,1923,177,27972
3,2020,2020-04-01,0,5,2,0,0,0,0,6,277,2,0,386,0,0,678
4,2020,2020-05-01,2,2,0,0,0,2,0,6,276,0,0,412,9,0,709


## Aviation Data

In [4]:
aviation_path = os.getcwd() + "/data/tourism/aviation_seats_flights_pic.xlsx"
aviation = pd.read_excel(aviation_path)
aviation.head(5)

,Country,ISO,Region,Date,Aircraft_type,Seats_arrivals_domestic,Seats_arrivals_interregional,Seats_arrivals_intraregional,Seats_arrivals_intl,Seats_arrivals_total,Available_seat_kilometers,Number_of_flights_domestic,Number_of_flights_interregional,Number_of_flights_intraregional,Number_of_flights_intl,Number_of_flights_total
0,Fiji,FJ,East Asia & Pacific,2019-01-01,passenger,839,273,3480,3753,4592,1.430416e+07,8,1,10,11,19
1,Fiji,FJ,East Asia & Pacific,2019-01-02,passenger,974,313,3471,3784,4758,1.495610e+07,8,1,10,11,19
2,Fiji,FJ,East Asia & Pacific,2019-01-03,passenger,1190,443,3675,4118,5308,1.592143e+07,10,2,12,14,24
3,Fiji,FJ,East Asia & Pacific,2019-01-04,passenger,831,586,3159,3745,4576,1.457334e+07,7,2,12,14,21
4,Fiji,FJ,East Asia & Pacific,2019-01-05,passenger,744,273,4752,5025,5769,1.773449e+07,7,1,12,13,20


In [8]:
fj_avi = (aviation[(aviation.ISO == "FJ") & (aviation.Aircraft_type == "passenger")]
          .reset_index()
          .drop("index", axis=1)
          [["Date", "Number_of_flights_intl", "Number_of_flights_total",
            "Seats_arrivals_intl", "Seats_arrivals_total"]])

dates = pd.DataFrame(pd.date_range(start="2020-01-01",
                                   end="2022-08-01"), columns=["Date"])

fj_avi = dates.merge(fj_avi, how="left", on="Date")
fj_avi["Date"] = pd.to_datetime(fj_avi["Date"])
fj_avi["Month"], fj_avi["Year"] = fj_avi["Date"].dt.month, fj_avi["Date"].dt.year

fj_avi_19_22 = fj_avi.groupby(by=["Year", "Month"]).sum().reset_index()
fj_merged = pd.concat([fj_avi_19_22, df], axis=1)
fj_merged

,Year,Month,Number_of_flights_intl,Number_of_flights_total,Seats_arrivals_intl,Seats_arrivals_total,year,dates,australia,newzealand,...,continentaleurope,japan,southkorea,china,india,hongkong,rest ofasia,pacificislands,others,total
0,2020,1,342.0,732.0,107570.0,142531.0,2020,2020-01-01,29886,11316,...,2802,726,316,4469,367,65,1238,5022,472,65386
1,2020,2,248.0,522.0,65187.0,90950.0,2020,2020-02-01,16690,8242,...,2524,1042,161,1661,469,35,1014,4685,347,46343
2,2020,3,173.0,405.0,45688.0,65205.0,2020,2020-03-01,11260,5520,...,1361,460,28,835,276,55,896,1923,177,27972
3,2020,4,9.0,58.0,1922.0,3869.0,2020,2020-04-01,0,5,...,0,0,6,277,2,0,386,0,0,678
4,2020,5,5.0,153.0,925.0,10087.0,2020,2020-05-01,2,2,...,2,0,6,276,0,0,412,9,0,709
5,2020,6,9.0,104.0,1526.0,6500.0,2020,2020-06-01,22,4,...,4,0,1,140,0,0,217,6,0,413
6,2020,7,11.0,63.0,1915.0,5698.0,2020,2020-07-01,16,11,...,20,0,4,261,0,0,333,31,4,711
7,2020,8,15.0,98.0,3190.0,12200.0,2020,2020-08-01,39,32,...,48,7,4,270,4,1,464,17,57,976
8,2020,9,31.0,128.0,7414.0,15168.0,2020,2020-09-01,39,55,...,39,2,6,336,21,0,396,70,2,1005
9,2020,10,44.0,145.0,10661.0,20673.0,2020,2020-10-01,33,33,...,16,2,12,335,2,0,580,59,9,1125


## Correlation Analysis

In [9]:
from scipy.stats import pearsonr


corr_seats_max, _ = pearsonr(fj_merged["Seats_arrivals_total"], fj_merged["total"])
corr_seat_flight_max, _ = pearsonr(fj_merged["Number_of_flights_total"], fj_merged["total"])
print(" Using the Maximum Values of the duplicated rows:  \n",
f"Pearson Correlation between FlightRadar's Seats Arrival and FJ's census data is{corr_seats_max: .4f}. \n",
f"Pearson Correlation between FlightRadar's # of Flights Arrival and FJ's census data is{corr_seat_flight_max: .4f}.")

# Correlation Matrix
cols = ["Seats_arrivals_intl", "Seats_arrivals_total",
        "Number_of_flights_total", "Number_of_flights_intl", "total"]
corr_matrix_max = fj_merged[cols].corr()
corr_matrix_max

 Using the Maximum Values of the duplicated rows:  
 Pearson Correlation between FlightRadar's Seats Arrival and FJ's census data is 0.7835. 
 Pearson Correlation between FlightRadar's # of Flights Arrival and FJ's census data is 0.7121.


,Seats_arrivals_intl,Seats_arrivals_total,Number_of_flights_total,Number_of_flights_intl,total
Seats_arrivals_intl,1.000000,0.990794,0.936246,0.979157,0.807567
Seats_arrivals_total,0.990794,1.000000,0.971783,0.980772,0.783485
Number_of_flights_total,0.936246,0.971783,1.000000,0.958268,0.712115
Number_of_flights_intl,0.979157,0.980772,0.958268,1.000000,0.786754
total,0.807567,0.783485,0.712115,0.786754,1.000000


### ADF tests

In [10]:
test_result = pd.DataFrame()
for col in cols:
    col_result = adf_test(fj_merged[col])
    col_result_df = pd.DataFrame(col_result)
    col_result_df.columns = [col]
    test_result = pd.concat([test_result, col_result_df.T], axis=0)
    
test_result

Results of Dickey-Fuller Test:
Test Statistic                 -2.120990
p-value                         0.236159
# Lags Used                    10.000000
Number of Observations Used    21.000000
Critical Value (1%)            -3.788386
Critical Value (5%)            -3.013098
Critical Value (10%)           -2.646397
dtype: float64
Results of Dickey-Fuller Test:
Test Statistic                 -1.165745
p-value                         0.688155
# Lags Used                     7.000000
Number of Observations Used    24.000000
Critical Value (1%)            -3.737709
Critical Value (5%)            -2.992216
Critical Value (10%)           -2.635747
dtype: float64
Results of Dickey-Fuller Test:
Test Statistic                 -3.333019
p-value                         0.013465
# Lags Used                     0.000000
Number of Observations Used    31.000000
Critical Value (1%)            -3.661429
Critical Value (5%)            -2.960525
Critical Value (10%)           -2.619319
dtype: float64
R

,Test Statistic,p-value,# Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%)
Seats_arrivals_intl,-2.120990,0.236159,10.0,21.0,-3.788386,-3.013098,-2.646397
Seats_arrivals_total,-1.165745,0.688155,7.0,24.0,-3.737709,-2.992216,-2.635747
Number_of_flights_total,-3.333019,0.013465,0.0,31.0,-3.661429,-2.960525,-2.619319
Number_of_flights_intl,-0.376402,0.913965,7.0,24.0,-3.737709,-2.992216,-2.635747
total,0.717172,0.990174,7.0,24.0,-3.737709,-2.992216,-2.635747
